In [22]:
import datetime
import inspect
import numpy as np
import pandas as pd

from pynwb import NWBHDF5IO, NWBFile
from pynwb.core import DynamicTableRegion, DynamicTable
from pynwb.device import Device

#from ndx_stimulation import StimulusFunctionTable
import os

from pynwb import register_class, load_namespaces
from hdmf.utils import docval, call_docval_func, getargs, get_docval


In [23]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [24]:
ndx_stimulation_specpath = '/Users/pam.baker/Documents/ndx_stim/ndx-stimulation/spec/ndx-stimulation.namespace.yaml'

# Load the namespace                                                                                                 
load_namespaces(ndx_stimulation_specpath)

{'ndx-stimulation': {'core': ('Container', 'DynamicTable'),
  'hdmf-common': ('VectorIndex', 'VectorData')}}

In [25]:
@register_class('StimulusFunctionTable', 'ndx-stimulation')
class StimulusFunctionTable(DynamicTable):   
    
    __columns__ = ( 
                    {'name':'function_name', 
                     'description': 'The names of the 1D stimulus waveforms.',
                     'required': True,
                     'index': False},
                    {'name':'float_parameters', 
                     'description': 'The names of the float parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True},
                    {'name':'function_parameters',
                     'description': 'The function parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True}
    )
    
    
    @docval(*get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        kwargs['name'] = ('Stimulus Function Table')
        kwargs['description'] = ('Table for storing ontologized 1D stimulus waveform metadata')
        call_docval_func(super().__init__, kwargs)
        

In [26]:
sft = StimulusFunctionTable()

float_params1 = [('amplitude', -0.110, 'V'), 
                 ('duration', 0.500, 's'),
                 ('start_time', 0.1, 's')
                ]

float_params2 = [('amplitude', -0.090, 'V'), 
                 ('duration', 1, 's'),
                 ('start_time', 0.1, 's')
                ]

wave1 = {'function_name':'sq', 'float_parameters': float_params1, 'function_parameters': float_params2}
wave2 = {'function_name':'sq', 'float_parameters': float_params2, 'function_parameters':[]}

sft.add_row(data = wave1)
sft.add_row(data = wave2)

In [6]:
pd.set_option("display.max_colwidth", 100)

display(sft.to_dataframe())

,function_name,float_parameters,function_parameters
id,,,
0,sq,"[(amplitude, -0.11, V), (duration, 0.5, s), (start_time, 0.1, s)]","[(amplitude, -0.09, V), (duration, 1, s), (start_time, 0.1, s)]"
1,sq,"[(amplitude, -0.09, V), (duration, 1, s), (start_time, 0.1, s)]",[]


In [7]:
#yay = sft.get('function_name')
#type(yay)

#sft.get('float_parameters').data[:]
sft['function_name'].data[:][0]
sft['float_parameters'].target[0][2]

'V'

### External Resources Table

- Resources: eg ontology we are referencing
- Entities: as defined by an entry in ontology/controlled terms
- Keys: reference from object into the ER tables that maps onto entities (naming need not be unique)
- Objects: thing we are ontologizing (eg ERT)


In [28]:
from hdmf.common import ExternalResources
from hdmf import Container, Data
import pandas as pd

In [27]:
er = ExternalResources(name='ExtResTable')

In [29]:
object_id = sft.object_id
print(object_id)

b160dba6-156c-4ff2-a461-50e6d179824d


In [30]:
# add_ref creates an entry across all tables (entities, keys, objects, resources)

er.add_ref(container = object_id, 
           field='', 
           key='sq',
           resource_name='stim_onto',
           resource_uri='stim_onto_uri',
           entity_id='sq',
           entity_uri='sq_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='duration',
           resource_name='stim_onto',
           resource_uri='stim_onto_uri',
           entity_id='sq_duration',
           entity_uri='sq_duration_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='amplitude',
           resource_name='stim_onto',
           resource_uri='stim_onto_uri',
           entity_id='sq_amplitude',
           entity_uri='sq_amplitude_uri'
          )

er.add_ref(container = object_id, 
           field='', 
           key='start_time',
           resource_name='stim_onto',
           resource_uri='stim_onto_uri',
           entity_id='sq_start_time',
           entity_uri='sq_start_time_uri'
          )


In [12]:
er.resources.to_dataframe()

,resource,resource_uri
0,stim_onto,stim_onto_uri
1,stim_onto,stim_onto_uri
2,stim_onto,stim_onto_uri
3,stim_onto,stim_onto_uri


In [20]:
er.entities.to_dataframe()

,keys_idx,resources_idx,entity_id,entity_uri
0,0,0,sq,sq_uri
1,1,1,sq_duration,sq_duration_uri
2,2,2,sq_amplitude,sq_amplitude_uri
3,3,3,sq_start_time,sq_start_time_uri


In [21]:
er.keys.to_dataframe()

,key
0,sq
1,duration
2,amplitude
3,start_time


In [33]:
k=er.add_key(key_name='sq')
er.keys.to_dataframe()

,key
0,sq
1,duration
2,amplitude
3,start_time
4,sq


In [34]:
key_object = er.get_key(key_name='sq')
print(key_object)

[<hdmf.common.resources.Key object at 0x15023fd90>, <hdmf.common.resources.Key object at 0x15023f510>]


### Write

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal

start_time = datetime(2017, 4, 3, 11, tzinfo=tzlocal())
create_date = datetime(2017, 4, 15, 12, tzinfo=tzlocal())


nwbfile = NWBFile('demonstrate caching', 'NWB456', start_time,
                  file_create_date=create_date)

nwbfile.add_acquisition(sft)

In [ ]:
# Write the SFT out to file

from pynwb import NWBHDF5IO

io = NWBHDF5IO('sft_cache_spec_example2.nwb', mode='w')
io.write(nwbfile)
io.close()

In [ ]:
# Reading in the file I just wrote

io = NWBHDF5IO('sft_cache_spec_example.nwb', mode='r', load_namespaces=True)
sft_nwbfile = io.read()

In [ ]:
# print(sft_nwbfile.acquisition)

sft_in = sft_nwbfile.get_acquisition()

In [ ]:
sft_in.to_dataframe()

In [ ]:
sft_in['float_parameters'].target[0][2]

In [ ]:
from random import random, randint

float_val = random()
function_index = randint(0, 10)

In [ ]:
print(float_val, function_index )

In [ ]:
        float_value = random()
        function_index = randint(0, 10)
        float_param = ('name', float_value, 'units')
        function_param = ('name', function_index, 'units')

        row = {
            'function_name':'function_name',
            'float_parameters': float_param,
            'function_parameters': function_param
            }

        stim_func_table = StimulusFunctionTable()
        stim_func_table.add_row(data=row)

In [ ]:
stim_func_table.to_dataframe()

In [ ]:
stim_func_table['function_parameters'].target[1]

In [ ]:
from random import random, randint

float_value = random()
function_index = randint(0, 10)
float_param = [('name', float_value, 'units'), ('name', float_value, 'units')]
function_param = [('name', function_index, 'units'), ('name', function_index, 'units')]

row = {
    'function_name':'function_name',
    'float_parameters': float_param,
    'function_parameters': function_param
    }

stim_func_table = StimulusFunctionTable()
stim_func_table.add_row(data=row)


In [ ]:
stim_func_table.add_row(data=row)


In [ ]:
stim_func_table.to_dataframe()

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal

start_time = datetime(2017, 4, 3, 11, tzinfo=tzlocal())
create_date = datetime(2017, 4, 15, 12, tzinfo=tzlocal())


nwbfile = NWBFile('demonstrate caching', 'NWB456', start_time,
                  file_create_date=create_date)

# path = '/Users/pam.baker/Documents/ndx_stim/ndx-stimulation/src/pynwb/tests/sft_cache_spec_example.nwb'

nwbfile.add_acquisition(stim_func_table)

io = NWBHDF5IO('sft_testy2_example.nwb', mode='w')
io.write(nwbfile)
io.close()

# with NWBHDF5IO(path, mode='w') as io:
#    io.write(nwbfile)